# Load environment variables and keys

In [144]:
from dotenv import dotenv_values
# specify the name of the .env file name 
env_name = "example.env"
config = dotenv_values(env_name)

In [145]:
import openai
openai.api_type = config["openai_api_type"] #"azure"
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_base'] #"https://synapseml-openai.openai.azure.com/"
openai.api_version = config['openai_api_version'] 
openai_deployment = config["openai_deployment_embedding"]
EMBEDDING_LENGTH = 1536

In [146]:
cogsearch_key = config["cogsearch_api_key"]
service_endpoint = config["cogsearch_endpoint"]
index_name = config["cogsearch_index_name"]


# Upload data to SQL DB

## Connect to database

For simplicity, we set `autocommit=True` in the pyodbc connection parameters, which allows us to execute `ALTER` statements. In a production scenario, setting `autocommit=True` is not recommended; instead, the `ALTER` statements can be executed in SSMS or similar.

In [147]:
import pyodbc

# Define Azure SQL database connection details
server = config["server"] 
database = config["database"] 
username = config["username"] 
password = config["password"] 
driver = '{ODBC Driver 18 for SQL Server}'

# Create a connection string
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Establish a connection to the Azure SQL database
conn = pyodbc.connect(conn_str, autocommit=True)
cursor = conn.cursor()

## Create a table in the database

We will create a new table "foodreview" and upload the data from a csv file. We include a primary key, which is necessary for change tracking.

In [148]:
table_name = "foodreview" 

# Drop previous table of same name if one exists
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
print("Finished dropping table (if existed)")

# Create a table
cursor.execute(f"""
               CREATE TABLE {table_name} 
               (Id int NOT NULL, 
               CONSTRAINT PK_{table_name}_Id PRIMARY KEY CLUSTERED (Id), 
               ProductId text, 
               UserId text, 
               ProfileName text, 
               HelpfulnessNumerator integer, 
               HelpfulnessDenominator integer, 
               Score integer, 
               Time bigint, 
               Summary text, 
               Text text);
               """)
print("Finished creating table")

# Create a index
cursor.execute(f"CREATE INDEX idx_Id ON {table_name}(Id);")
print("Finished creating index")

Finished dropping table (if existed)
Finished creating table
Finished creating index


## Enable change tracking

This allows the us to automatically update the index when changes are made to the data.

In [149]:
try:
    cursor.execute(f"ALTER DATABASE {database} SET CHANGE_TRACKING = ON (CHANGE_RETENTION = 2 DAYS, AUTO_CLEANUP = ON)")
except Exception as e:
    print(e)

('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Change tracking is already enabled for database 'test_vector_notebook'. (5088) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]ALTER DATABASE statement failed. (5069)")


In [150]:
try:
    cursor.execute(f"ALTER TABLE {table_name} ENABLE CHANGE_TRACKING WITH (TRACK_COLUMNS_UPDATED = ON)")
except Exception as e:
    print(e)

## Upload data

We split our data into an initial dataset and an extra dataset (the extra dataset will be used demo the change tracking feature later).

In [151]:
## Load Data
import numpy as np
import pandas as pd
df_all = pd.read_csv('../../DataSet/Reviews_small.csv')

# Split data into 'initial' and 'extra' data
df_initial = df_all[:50].copy()
df_extra = df_all[50:].copy().reset_index(drop = True)

In [152]:
## Upload initial data
df = df_initial 

# Specify the batch size
batch_size = 30

# Split the dataframe into batches
batches = [df[i:i + batch_size] for i in range(0, len(df), batch_size)]

#Iterate over each batch and insert the data into the database
for batch in batches:
    # Convert the batch dataframe to a list of tuples for bulk insertion
    rows = [tuple(row) for row in batch.itertuples(index=False)]
    
    # Define the SQL query for bulk insertion
    query = f"INSERT INTO {table_name} (Id, ProductId, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary, Text) \
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.executemany(query, rows)

## Example query

In [153]:
# Execute the SELECT statement
try:
    cursor.execute(f"SELECT count(Id) FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing SELECT statement: {e}")

(50, )


## Commit changes

In [154]:
cursor.commit()
cursor.close()

# Set up data source connection in Cog Search

## Import needed CogSearch functions

In [155]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType,
    RawVectorQuery,
    VectorizableTextQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (  
    AzureOpenAIEmbeddingSkill,  
    AzureOpenAIParameters,  
    AzureOpenAIVectorizer,  
    ExhaustiveKnnParameters,  
    ExhaustiveKnnVectorSearchAlgorithmConfiguration,
    FieldMapping,  
    HnswParameters,  
    HnswVectorSearchAlgorithmConfiguration,  
    IndexProjectionMode,  
    InputFieldMappingEntry,  
    OutputFieldMappingEntry,  
    PrioritizedFields,    
    SearchField,  
    SearchFieldDataType,  
    SearchIndex,  
    SearchIndexer,  
    SearchIndexerDataContainer,  
    SearchIndexerDataSourceConnection,  
    SearchIndexerIndexProjectionSelector,  
    SearchIndexerIndexProjections,  
    SearchIndexerIndexProjectionsParameters,  
    SearchIndexerSkillset,  
    SemanticConfiguration,  
    SemanticField,  
    SemanticSettings,  
    SplitSkill,  
    SqlIntegratedChangeTrackingPolicy,
    VectorSearch,  
    VectorSearchAlgorithmKind,  
    VectorSearchAlgorithmMetric,  
    VectorSearchProfile,  
)  

## Create data source connection

In [156]:
ds_conn_str = f'Encrypt=True;TrustServerCertificate=False;Connection Timeout=30;Server=tcp:{server};Database={database};User ID={username};Password={password};'

cogsearch_credential = AzureKeyCredential(cogsearch_key)
ds_client = SearchIndexerClient(service_endpoint, cogsearch_credential)
container = SearchIndexerDataContainer(name=table_name)
#"#Microsoft.Azure.Search.SqlIntegratedChangeTrackingPolicy"
change_detection_policy = SqlIntegratedChangeTrackingPolicy()

data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-azuresql-connection",
    type="azuresql",
    connection_string=ds_conn_str,
    container=container,
    data_change_detection_policy=change_detection_policy
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

Data source 'amazon-review-jordan-v1-azuresql-connection' created or updated


# Set up automatic indexing + vectorization

## Create index 

In [157]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=cogsearch_credential)

fields = [
    SearchField(name="Id", type=SearchFieldDataType.String, key=True,
                sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),
    SearchField(name="parent_id", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
                vector_search_dimensions=EMBEDDING_LENGTH, vector_search_profile="my-vector-search-profile")
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-hnsw-config",
            kind=VectorSearchAlgorithmKind.HNSW
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="my-vector-search-profile",
            algorithm="my-hnsw-config",
            vectorizer="my-openai"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="my-openai",
            kind="azureOpenAI",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=openai.api_base,
                deployment_id=openai_deployment,
                api_key=openai.api_key
            )
        )  
    ]  
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        prioritized_content_fields=[SemanticField(field_name="Id")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f'{result.name} created')


amazon-review-jordan-v1 created


## Create skillset

This section of the code sets up the automatic chunking of text and automatic vectorization of the text after chunking. We recommend the following resources to learn more about the process and how one can adapt it to different applications:
* [Overview of indexers](https://learn.microsoft.com/en-us/azure/search/search-indexer-overview)
* [Skill context and input annotation language](https://learn.microsoft.com/en-us/azure/search/cognitive-search-skill-annotation-language)
* [Reference inputs and outputs in skillsets](https://learn.microsoft.com/en-us/azure/search/cognitive-search-concept-annotations-syntax)

In [158]:
# Create a skillset  
skillset_name = f"{index_name}-skillset"  

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=300,  
    page_overlap_length=20,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/Text"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ]  
)

embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_uri=openai.api_base,  
    deployment_id=openai_deployment,  
    api_key=openai.api_key,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="vector")  
    ]  
)  

index_projections = SearchIndexerIndexProjections(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=index_name,  
            parent_key_field_name="parent_id",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                InputFieldMappingEntry(name="vector", source="/document/pages/*/vector")
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
)  

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents and generating embeddings",  
    skills=[split_skill, embedding_skill],
    index_projections=index_projections  
)
  
client = SearchIndexerClient(service_endpoint, cogsearch_credential)  
client.create_or_update_skillset(skillset)  
print(f' {skillset.name} created')

 amazon-review-jordan-v1-skillset created


## Create indexer

TODO: Check connection; make timeout logic

In [159]:
# Create an indexer  
indexer_name = f"{index_name}-indexer"  

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to chunk documents and generate embeddings",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name
)  
  
indexer_client = SearchIndexerClient(service_endpoint, cogsearch_credential)
indexer_result = indexer_client.create_or_update_indexer(indexer)  

# Run the indexer  
indexer_client.run_indexer(indexer_name)
print(f' {indexer_name} created')

 amazon-review-jordan-v1-indexer created


In [160]:
# Get the status of the indexer  
indexer_status = indexer_client.get_indexer_status(indexer_name)
print(f"Indexer status: {indexer_status.status}")

Indexer status: running


In [161]:
# Allow some time for the indexer to process the data
import time
time.sleep(30)

# Use vector search for sample application

## Perform queries

In [162]:
user_query = "Canned dog food"

In the following output, we find the top 3 chunks that are most relevant to the user's query.

Feel free to retry the following cell in case of an empty response or a 429 error. An empty response probably indicates that the chunking/embedding process has not finished yet. A 429 error means there have been too many requests to the OpenAI embedding service and should go away on retrying.

In [163]:
search_client = SearchClient(service_endpoint, index_name, credential=cogsearch_credential)
vector_query = VectorizableTextQuery(text=user_query, k=3, fields="vector", exhaustive=True)
# Use the query below to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(user_query), k=3, fields="vector")
  
results = search_client.search(
    search_text=None,  
    vector_queries= [vector_query],
    select=["Id", "parent_id", "chunk"],
    top=3
)

for result in results: 
    print(f"Chunk id: {result['Id']}")
    print(f"Parent Id: {result['parent_id']}")
    print(f"Search score: {result['@search.score']}")
    print(f"Text chunk: {result['chunk']}")
    print("-----")


Chunk id: 7f37f895690c_1_pages_0
Parent Id: 1
Search score: 0.880716
Text chunk: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
-----
Chunk id: 675e8e92bf89_10_pages_0
Parent Id: 10
Search score: 0.84844846
Text chunk: This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.
-----
Chunk id: 98a114de2791_33_pages_2
Parent Id: 33
Search score: 0.8391927
Text chunk: pack taste good.  It can be prepared in the microwave or by adding boiling water so it is convenient in the extreme when time is an issue.<br /><br />McCann's use of actual cane sugar instead of high fructose corn syrup helped me decide to buy this product.
-----


## Generate GPT Response

### Prompt creation

In [164]:
# create a prompt template 
template = """
    The user's question is: {query}
    The context for this question is:{context}
    Answer the question based on the provided context. Your answer should summarize one or more relevant
    reviews, and be sure to mention the corresponding product id's.
    Question: {query}
    Answer: """

In [165]:
# create the context from the search response (requires regenerating results)
results = search_client.search(
    search_text=None,  
    vector_queries= [vector_query],
    select=["Id", "parent_id", "chunk"],
    top=3
)

context = "These three product reviews are relevant to the user's question: \n"

for result in results:
    context += f"Product id: {result['parent_id']}. Review text: {result['chunk']}"
    context += "\n"
    
print(context)

These three product reviews are relevant to the user's question: 
Product id: 1. Review text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Product id: 10. Review text: This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.
Product id: 33. Review text: pack taste good.  It can be prepared in the microwave or by adding boiling water so it is convenient in the extreme when time is an issue.<br /><br />McCann's use of actual cane sugar instead of high fructose corn syrup helped me decide to buy this product.



In [166]:
prompt = template.format(context=context, query=user_query)
print(prompt)


    The user's question is: Canned dog food
    The context for this question is:These three product reviews are relevant to the user's question: 
Product id: 1. Review text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Product id: 10. Review text: This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.
Product id: 33. Review text: pack taste good.  It can be prepared in the microwave or by adding boiling water so it is convenient in the extreme when time is an issue.<br /><br />McCann's use of actual cane sugar instead of high fructose corn syrup helped me decide to buy this product.

    Answer the question based on the provided context. Your answer should summarize one or more relevant


### Call to OpenAI

In [167]:
response = openai.Completion.create(
    engine= config["openai_deployment_completion"],
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=1,
)

print(response['choices'][0]['text'])

 Product id 1 and 10 both show that the canned dog food they have bought is of high quality and provides a healthy meal for their dogs. Product id 33 mentions McCann's use of actual cane sugar instead of high fructose corn syrup, which makes the product attractive to buyers.
